In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import scipy
import scipy.stats as stats
import os

In [2]:
# project files

from utils.variables import pack, unpack
from utils.display import table, tabulate
from utils.files import download_sheet, save, load
from utils.hash import digest


from fit_options import fit_options
from game_model import game
from rp_model import compute_rp, make_precomputed_columns
from initial_guess import make_initial_guess


In [3]:
# stuff for display

from IPython.lib.pretty import pretty, pprint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [4]:
# Load previous data

data = pd.read_pickle(fit_options.data_file)
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,FreqL,Inv,Berry1,BerryL,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,BerryD,IngD,Dupes,Amnt,Ing2P,Help skill bonus,RP Multiplier
count,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.000000,6326.00000,6326.000000,6326.000000,6326.0,6326.000000,6326.000000,6326.000000,6326.000000
mean,10.600221,604.489251,1.490831,604.527087,0.037836,3957.192539,3807.605275,12.229529,28.040942,37.740120,108.840500,1.017855,0.988552,0.994790,1.000917,1.005375,0.207831,32.57993,55.241385,148.778375,1.0,0.072400,2.962536,0.987673,1.028040
std,7.144162,388.638606,0.982291,388.693660,0.962510,962.301896,1023.063474,3.385034,2.611147,8.005598,15.997332,0.303530,0.059303,0.044976,0.117870,0.118527,0.044906,36.08216,27.687375,60.605390,0.0,0.511503,20.447361,0.037609,0.070145
min,1.000000,245.000000,1.000000,245.060000,-11.980000,2200.000000,1489.118400,7.000000,24.000000,24.000000,90.000000,0.520000,0.890000,0.920000,0.800000,0.800000,0.130000,5.88000,25.000000,90.000000,1.0,0.000000,0.000000,0.790000,1.000000
25%,5.000000,355.000000,1.000000,354.635000,-0.280000,3300.000000,2955.960000,10.000000,26.000000,32.000000,98.000000,0.800000,1.000000,1.000000,1.000000,1.000000,0.180000,12.81000,35.000000,101.000000,1.0,0.000000,0.000000,1.000000,1.000000
50%,10.000000,462.000000,1.000000,461.845000,-0.010000,3900.000000,3792.400000,12.000000,28.000000,36.000000,103.000000,0.940000,1.000000,1.000000,1.000000,1.000000,0.200000,19.98000,45.000000,121.000000,1.0,0.000000,0.000000,1.000000,1.000000
75%,14.000000,712.000000,2.000000,711.970000,0.270000,4500.000000,4455.000000,14.000000,31.000000,42.000000,115.000000,1.210000,1.000000,1.000000,1.000000,1.000000,0.240000,43.98000,68.000000,202.000000,1.0,0.000000,0.000000,1.000000,1.000000
max,42.000000,3440.000000,6.000000,3440.430000,10.460000,6300.000000,6916.140000,23.000000,35.000000,77.000000,151.000000,2.410000,1.120000,1.080000,1.200000,1.200000,0.420000,413.95000,222.000000,511.000000,1.0,8.000000,342.000000,1.000000,1.440000


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
0,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.42,0.42,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,3700.0,3276.72,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,1.09,0.90,1.00,1.0,0.8,0.26,22.76,538.36,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
1,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,530.70,-0.30,Neutral,ArbokNeutral81,-,-,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.0,1.0,0.26,22.76,531.00,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
2,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.09,0.09,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.2,1.0,0.26,22.76,530.61,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
3,Arcanine,5.0,958.0,Calm,3.0,,957.90,-0.10,Calm,ArcanineCalm53,Main Skill Chance,Speed of Help,2500.0,2728.00,Fire,Skills,Extra Helpful S,16.0,27.0,31,Fiery Herb,130.0,1.31,1.11,1.00,1.0,1.2,0.14,84.50,937.84,ArcanineNeutral53,31.0,130.0,1,True,,,,0.0,0.0,1.0,1.0
4,Arcanine,3.0,715.0,Lax,2.0,pokemon sleep general,714.77,-0.23,Lax,ArcanineLax32,Energy Recovery,Main Skill Chance,2500.0,2490.00,Fire,Skills,Extra Helpful S,16.0,27.0,29,Fiery Herb,130.0,1.44,1.00,1.08,1.0,0.8,0.14,61.21,750.21,ArcanineNeutral32,29.0,130.0,1,True,,,,0.0,0.0,1.0,1.0


In [5]:
# Load previous fit

x0, unpack_info = pack(make_initial_guess())

hash_value = digest(data, x0)
filename = fit_options.result_file(hash_value)

opt = load(filename)
sol = unpack(opt.x, unpack_info)

opt

    message: The maximum number of function evaluations is exceeded.
    success: False
     status: 0
          x: [ 2.521e-01  2.502e-01 ...  2.199e-01  2.199e-01]
       cost: 4144.405211938394
       grad: [ 1.476e+03  4.234e+03 ... -9.749e+03 -7.575e+04]
 optimality: 9211562.42645229
       nfev: 200
       njev: 14

In [6]:
# Helpers

def truncated_normal_sample(size, mu, sigma, lower, upper):
    return stats.truncnorm.rvs( (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma, size=size)

def round_noise(size):
    eps = 1e-6
    return truncated_normal_sample(size, mu=0.0, sigma=0.25, lower= -0.5+eps, upper= 0.5-eps)

def harmonic(a,b):
    return 2.0 / ( 1.0 / a + 1.0 / b )

In [7]:
# In the boostrap method,
# We create N synthetic datasets that are likely to represent the population
# Then we redo the fit on those, starting from the best fit in initial dataset.
# Finally, we collect statistics on the fitted variables between runs

# To create the synthetic datasets, we use sampling with replacement (allow duplicate)
# Because some Pokémon are rare, we use stratified sampling

stratas = data.groupby(['Pokemon'], group_keys=False)
target_group_size =  stratas.size().quantile(q=0.75)
    

n_boostraps = 50
all_opt_x = [opt.x]

In [8]:
cache_fit = True
boostrap_filename = f"./results/bootstrap-fit-{hash_value}.pickle"

if cache_fit and os.path.isfile(boostrap_filename):

    all_opt_x = load(boostrap_filename)
    print("Loaded from cache")

else:

    # WARNING: RUNNING THE OPTIMISATION N TIMES IS VERY LONG.
    #
    # YOU CAN INTERRUPT THE KERNEL (STOP BUTTON)
    # AND RUN THE NEXT CELLS IF YOU WANT TO SEE CURRENT PROGRESS
    #
    # THE COLLECTION `all_opt_x` IS DEFINED ABOVE THIS CELL
    # SO YOU CAN RESUME ADDING MORE RUNS

    while( len(all_opt_x) < n_boostraps ):

        print( "Boostrap run " + str(len(all_opt_x)) )

        # To create the synthetic datasets, we use sampling with replacement (allow duplicate)

        # We'll introduce a correction toward more equal sample size between Pokémon
        # We can motivate that by the fact stratified sampling should use population proportions
        # instead of current data proportions

        resampled = stratas.apply(lambda x: x.sample( round( harmonic(len(x),target_group_size) ), replace=True, ignore_index=True))

        # Add small noise to RP
        # We do so to simulate some unknown value that would round() to current RP

        referenceRP = resampled["RP"].to_numpy()
        referenceRP += round_noise(len(resampled))

        # Compute per sample information about help time, nature, sub-skills etc
        recomputed = make_precomputed_columns(resampled)

        # Put the pieces together

        def residual(x):
            return referenceRP - compute_rp(x, resampled, recomputed, unpack_info)

        fit_options.soft_round.exact = False
        fit_options.soft_round.alpha = 6

        # Redo the fit, starting from the optimal we found on current data
        opt2 = scipy.optimize.least_squares(residual, opt.x, **fit_options.least_squares_kwargs)

        # Collect results for stats
        all_opt_x.append(opt2.x)

    # Save
    save(boostrap_filename, all_opt_x)

Boostrap run 1
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.1411e+03                                    4.64e+07    
       1             11         4.8954e+03      2.46e+02       1.30e-03       5.51e+07    
       2             12         4.8058e+03      8.96e+01       1.30e-03       2.75e+07    
       3             13         4.8052e+03      5.65e-01       1.30e-03       3.78e+07    
       4             14         4.8038e+03      1.46e+00       3.26e-04       3.63e+07    
       5             15         4.7875e+03      1.63e+01       8.15e-05       3.76e+07    
       6             16         4.7771e+03      1.03e+01       8.15e-05       9.33e+07    
       7             17         4.7519e+03      2.52e+01       8.15e-05       3.30e+07    
       8             18         4.7342e+03      1.77e+01       8.15e-05       3.53e+07    
       9             19         4.7287e+03      5.54e+00       8.15e-05    

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: overflow encountered in power
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:154: RuntimeWarning: invalid value encountered in scalar divide
  ratio = phi / phi_prime


      33             200        4.3522e+03      0.00e+00       0.00e+00       4.32e+06    
The maximum number of function evaluations is exceeded.
Function evaluations 200, initial cost 4.8611e+03, final cost 4.3522e+03, first-order optimality 4.32e+06.
Boostrap run 5
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.3883e+03                                    2.65e+07    
       1             10         4.1720e+03      2.16e+02       5.22e-03       5.74e+07    
       2             11         4.0509e+03      1.21e+02       1.30e-03       3.86e+07    
       3             12         3.9470e+03      1.04e+02       2.61e-03       1.49e+07    
       4             14         3.9251e+03      2.19e+01       6.52e-04       2.45e+06    
       5             15         3.9037e+03      2.14e+01       1.63e-04       2.58e+07    
       6             16         3.8985e+03      5.20e+00       1.63e-04       5.70e+07    
   

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: overflow encountered in power
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:154: RuntimeWarning: invalid value encountered in scalar divide
  ratio = phi / phi_prime


      38             200        4.3408e+03      0.00e+00       0.00e+00       3.25e+06    
The maximum number of function evaluations is exceeded.
Function evaluations 200, initial cost 4.8250e+03, final cost 4.3408e+03, first-order optimality 3.25e+06.
Boostrap run 7
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.2868e+03                                    6.20e+07    
       1             11         4.9563e+03      3.31e+02       1.30e-03       3.39e+07    
       2             12         4.8692e+03      8.70e+01       1.30e-03       4.34e+07    
       3             13         4.8580e+03      1.12e+01       3.26e-04       8.02e+07    
       4             14         4.8213e+03      3.68e+01       8.15e-05       7.15e+07    
       5             20         4.8143e+03      6.92e+00       1.59e-07       2.49e+06    
       6             21         4.8142e+03      1.55e-01       3.18e-07       5.13e+06    
   

...

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: overflow encountered in power
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:154: RuntimeWarning: invalid value encountered in scalar divide
  ratio = phi / phi_prime


      11             200        4.7577e+03      0.00e+00       0.00e+00       3.82e+05    
The maximum number of function evaluations is exceeded.
Function evaluations 200, initial cost 5.3176e+03, final cost 4.7577e+03, first-order optimality 3.82e+05.
Boostrap run 28
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.0966e+03                                    4.01e+07    
       1             11         4.9279e+03      1.69e+02       1.30e-03       8.52e+06    
       2             13         4.9221e+03      5.87e+00       3.26e-04       4.58e+07    
       3             14         4.9046e+03      1.74e+01       8.15e-05       5.45e+07    
       4             15         4.8867e+03      1.79e+01       8.15e-05       7.87e+07    
       5             16         4.8796e+03      7.09e+00       8.15e-05       4.87e+07    
       6             17         4.8610e+03      1.86e+01       2.04e-05       1.52e+07    
  

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: overflow encountered in power
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:154: RuntimeWarning: divide by zero encountered in scalar divide
  ratio = phi / phi_prime
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:166: RuntimeWarning: divide by zero encountered in scalar divide
  p *= Delta / norm(p)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:166: RuntimeWarning: invalid value encountered in multiply
  p *= Delta / norm(p)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\common.py:115: RuntimeWarning: invalid value encountered in scalar divide
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm


      10             200        4.6423e+03      0.00e+00       0.00e+00       1.99e+06    
The maximum number of function evaluations is exceeded.
Function evaluations 200, initial cost 5.4704e+03, final cost 4.6423e+03, first-order optimality 1.99e+06.
Boostrap run 32
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.7436e+03                                    3.19e+07    
       1             10         5.4367e+03      3.07e+02       5.22e-03       8.19e+05    
       2             11         5.1855e+03      2.51e+02       1.30e-03       9.35e+07    
       3             19         5.1755e+03      1.01e+01       1.59e-07       2.57e+07    
       4             20         5.1752e+03      2.34e-01       3.18e-07       2.50e+07    
       5             21         5.1745e+03      7.25e-01       7.96e-08       4.84e+06    
       6             22         5.1744e+03      7.18e-02       7.96e-08       4.53e+06    
  


KeyboardInterrupt



In [ ]:
# Stats
# We'll use the median as estimate for the mean.
# And 1.4826 * < median absolute deviation from the median > as estimate for std

center = np.median(all_opt_x, axis=0)

interval_std = 1.96 * np.std( all_opt_x, axis=0) # 95% confidence region
interval_mad = 3 * np.median( list(map(lambda x: np.abs(center-x), all_opt_x)), axis=0) # 95% confidence region

sol_center = unpack(center, unpack_info)
sol_interval_std = unpack(interval_std, unpack_info)
sol_interval_mad = unpack(interval_mad, unpack_info)

# Pretty display
pd.DataFrame({
    
    "Pokemon":game.pokedex.data["Pokemon"], 
    
    "ing%": sol_center["Pokemons ing fractions"]*100.0,
    #"conf (ing)": sol_interval_std["Pokemons ing fractions"]*100.0,
    "conf (ing)*": sol_interval_mad["Pokemons ing fractions"]*100.0,

    "skill% * skillValue": sol_center["Pokemons skill products"], 
    #"conf (skill)": sol_interval_std["Pokemons skill products"],
    "conf (skill)*": sol_interval_mad["Pokemons skill products"],


}).set_index("Pokemon")

In [ ]:
# Explain what we did with the re-sampling target size
# pd.DataFrame({'before': stratas.size(), 'after': stratas.apply(lambda x: round( harmonic(len(x), target_group_size) )) })